Topics
--------
1. TensorFlow Graphs
2. TensorFlow Sessions
3. Adding Inputs with Placeholder nodes
4. Variables
5. Organizing your graph with name scopes


## 1. TensorFlow graphs

In the previous tutorial, we've only referenced *the graph* as some sort of abstract, and we havent questioned how Operations are automatically attached to a graph when we use it. 

Now let's take a look at the [TensorFlow Graph Object](https://www.tensorflow.org/versions/master/api_docs/python/framework/#Graph), learn how to create more of them, use multiple graphs in conjunction with one another.

In [1]:
import tensorflow as tf

In [2]:
# Create a new graph:
g = tf.Graph()

Once we have our Graph initialized, we can add Operations to it by using the ```python Graph.as_default()``` method to access its context manager.

In [3]:
with g.as_default():
    # Create operations as usual; they will be added to graph 'g'
    a = tf.mul(2, 5)

We don't need to specify the graph we'd like to add our Ops. As a convenience, TensorFlow automatically creates a *Graph* when the library is  loaded and assigns it to be the default. Thus, any operations, tensors, etc. define outside of a ```python Graph.as_default()``` context manager will automatically be placed in the default graph:

In [4]:
# Placed in the default graph
in_default_graph = tf.add(2, 3)

# Placed in graph 'g'
with g.as_default():
    in_graph_g = tf.mul(2, 3)
    
# We are no longer in the 'with' block, so this is placed in the default 
# graph
also_in_default_graph = tf.sub(10, 5)


If you'd like to get a handle to the default graph, use the ```python tf.get_default_graph()``` function: ```default_graph = tf.get_default_graph()```

- In most TensorFlow programs, we only ever deal with the default graph. However, creating multiple graphs can be useful if you are defining multiple models that do not have interdependencies. When defining